# Opisivanje slike sa RNN
U ovoj vježbi ćete implementirati vanila rekurentne neuralne mreže i iskoristiti ih da se trenira model koji generiše opise slika.

In [ ]:
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from funkcije.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from funkcije.rnn_layers import *
from funkcije.captioning_solver import CaptioningSolver
from funkcije.classifiers.rnn import CaptioningRNN
from funkcije.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from funkcije.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ vraća relativnu grešku """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

## Instaliranje h5py
COCO dataset će biti korišten koji se nalazi u HDF5 formatu. Da biste učitali HDF5 fajlove, potrebno je da instalirate `h5py` Python pakaet. Iz komandne linije, pokrenite:<br/>
`pip install h5py`  <br/>
Ako dobijete grešku u dozvoli, biće potrebno da komandu pokrenete kao root:<br/>
```sudo pip install h5py```

Takođe možete pokrenuti komandu direktno iz Jupyter notebook dodavši karakter "!" kao prefiks komandi:

In [ ]:
!pip install h5py

# Microsoft COCO
Za ovu vježbu koristićete 2014 izdanje [Microsoft COCO dataset](http://mscoco.org/) koji je postao standard za testiranje opisivanja slika. Dataset se sastoji od 80 000 trening slika i 40000 validacionih slika, svakoj pribilježeno 5 titlova koji su napisani od strane radnika na Amazon Mechanical Turk.

Trebalo je da ste već skinuli podatke u direktorijumu `funkcije/datasets` i pokrenuti skriptu `get_assignment3_data.sh`. Ako do sada niste to uradili, sada pokrenite skriptu. Napomena: COCO podaci su ~1GB.

Mi smo obradili podatke i izvukli karakteristike za vas. 

Sirove slike zauzimaju previše prostora (blizu 20GB) tako da ih nismo uključili u skidanje. Međutim sve slike su uzete sa Flickr, i URL-ovi trening i validacionih slika su smješteni u fajlovima `train2014_urls.txt` i `val2014_urls.txt` respektivno. Ovo vam dozvoljava da skinete slike u hodu za vizuelizaciju. Kako su slike skinute u hodu, **morate biti konektovani na internetu da biste vidjeli slike**.

Bavljenje stringovima je neefikasno, pa ćemo raditi sa kodiranom verzijom opisa. Svakoj riječi je dodijeljen integer ID, dozvoljavajući nam da predstavimo opis kao sekvencu integers. Mapiranje između integers ID-ova i riječi je u fajlu `coco2014_vocab.json`, i možete koristiti funkciju `decode_captions` iz fajla `funkcije/coco_utils.py` da pretvorite numpx nizove ID-ova u stringove. 

Postoji par specijalnih tokena koje smo dodali u vokabularu. Na početku svakog opisa smo dodali `<START>` token i na kraju `<END>` token. Rijetke riječi su zamijenjene sa `<UNK>` tokenom ("unknown"). Kako želimo da treniramo minibatches koji sadrže opise različitih težina, proširujemo kratke opise sa `<NULL>` tokenom posle `<END>` tokena i ne računamo funkciju cilja ili gradijente za `<NULL>` tokene. S obzirom da specijalni tokeni mogu prouzrokavati dosta glavobolje, mi smo se pobrinuli o svim detaljima implementacije vezane za specijalne tokene.

Možete učitati sve MS-COCo podatke (opise, karakteristike, URL-ove i vokabular) koristeći `load_coco_data` funkciju iz fajla `funkcije/coco_utils.py`. Pokrenite sledeću ćeliju:

In [ ]:
# Učitavanje COCO podataka sa diska; ovo vraća rečnik
# Mi ćemo raditi sa dimenziono smanjenim karakteristikama, ali osjećajte 
# se slobodnim da eksperimentišete sa originalnim karakteristikama mijenjajući indikator ispod. 

data = load_coco_data(pca_features=True)

# Štampajte sve ključeve i vrijednosti iz rečnika podataka
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

## Pogledajte podatke
Uvijek je dobra ideja pogledati primjere iz dataset-a prije rada sa njima. 

Možete koristiti `sample_coco_minibatch` funkciju iz fajla `funkcije/coco_utils.py` da uzmete uzorke podataka iz strukture podataka iz `load_coco_data`. Pokrenite sledeću ćeliju da prikažete uzorak slika i njihovih opisa. Pokrećući ih više puta i gledajući rezultate će vam pomoći da steknete utisak o dataset-u.

Primijetite da dekodiramo opise koristeći `decode_captions` funkciju i da skidamo slike u hodu koristeći Flickr URL, tako da **morate biti povezani na internetu da biste vidjeli  slike**.

In [ ]:
batch_size = 3

captions, features, urls = sample_coco_minibatch(data, batch_size=batch_size)
for i, (caption, url) in enumerate(zip(captions, urls)):
    plt.imshow(image_from_url(url))
    plt.axis('off')
    caption_str = decode_captions(caption, data['idx_to_word'])
    plt.title(caption_str)
    plt.show()

# Rekurentne Neuralne Mreže
Koristićemo modele jezika rekurentnih neuralnih mreža za opisivanje slika. Fajl `funkcije/rnn_layers.py` sadrži implementacije različitih slojeva koji su potrebni za rekurentne neuralne mreže, a fajl `funkcije/classifiers/rnn.py` koristi te slojeve za implementaciju modela opisivanja slika. 

Prvo ćemo implementirati različite tipove RNN slojeva u `funkcije/rnn_layers.py`.

# Vanila RNN: korak unaprijed
Otvorite fajl `funkcije/rnn_layers.py`. Ovaj fajl implementira prolaze unaprijed i unazad za različite tipove slojeva koji se često koriste u rekurentnim neuralnim mrežama.

Prvo implementirajte funkciju `rnn_step_forward` koja implementira prolaz unaprijed za jedan vremenski korak vanila rekurentne neuralne mreže. Nakon toga pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebate da vidite grešku reda `e-8` ili manje.

In [ ]:
N, D, H = 3, 10, 4

x = np.linspace(-0.4, 0.7, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.2, 0.5, num=N*H).reshape(N, H)
Wx = np.linspace(-0.1, 0.9, num=D*H).reshape(D, H)
Wh = np.linspace(-0.3, 0.7, num=H*H).reshape(H, H)
b = np.linspace(-0.2, 0.4, num=H)

next_h, _ = rnn_step_forward(x, prev_h, Wx, Wh, b)
expected_next_h = np.asarray([
  [-0.58172089, -0.50182032, -0.41232771, -0.31410098],
  [ 0.66854692,  0.79562378,  0.87755553,  0.92795967],
  [ 0.97934501,  0.99144213,  0.99646691,  0.99854353]])

print('next_h greška: ', rel_error(expected_next_h, next_h))

# Vanila RNN: korak unazad
U fajlu `funkcije/rnn_layers.py` implementirajte `rnn_step_backward` funkciju. Nakon toga pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebali biste da vidite grešku reda `e-8` ili manje. 

In [ ]:
from funkcije.rnn_layers import rnn_step_forward, rnn_step_backward
np.random.seed(231)
N, D, H = 4, 5, 6
x = np.random.randn(N, D)
h = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnn_step_forward(x, h, Wx, Wh, b)

dnext_h = np.random.randn(*out.shape)

fx = lambda x: rnn_step_forward(x, h, Wx, Wh, b)[0]
fh = lambda prev_h: rnn_step_forward(x, h, Wx, Wh, b)[0]
fWx = lambda Wx: rnn_step_forward(x, h, Wx, Wh, b)[0]
fWh = lambda Wh: rnn_step_forward(x, h, Wx, Wh, b)[0]
fb = lambda b: rnn_step_forward(x, h, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dnext_h)
dprev_h_num = eval_numerical_gradient_array(fh, h, dnext_h)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dnext_h)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dnext_h)
db_num = eval_numerical_gradient_array(fb, b, dnext_h)

dx, dprev_h, dWx, dWh, db = rnn_step_backward(dnext_h, cache)

print('dx greška: ', rel_error(dx_num, dx))
print('dprev_h greška: ', rel_error(dprev_h_num, dprev_h))
print('dWx greška: ', rel_error(dWx_num, dWx))
print('dWh greška: ', rel_error(dWh_num, dWh))
print('db greška: ', rel_error(db_num, db))

# Vanila RNN: unaprijed
Sada kada ste implementirali prolaze unaprijed i unazad za jedan vremenski korak vanila RNN, kombinovaćete te djelove da biste implementirali RNN koja prolazi kroz cijelu sekvencu podataka.

U fajlu `funkcije/rnn_layers.py`, implementirajte funkciju `rnn_forward`. Ovo trebate implementirati koristeći `rnn_step_forward` funkciju koju ste definisali iznad. Nakon toga pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebali biste vidjeti grešku reda `e-7` ili manje.

In [ ]:
N, T, D, H = 2, 3, 4, 5

x = np.linspace(-0.1, 0.3, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.3, 0.1, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.4, num=D*H).reshape(D, H)
Wh = np.linspace(-0.4, 0.1, num=H*H).reshape(H, H)
b = np.linspace(-0.7, 0.1, num=H)

h, _ = rnn_forward(x, h0, Wx, Wh, b)
expected_h = np.asarray([
  [
    [-0.42070749, -0.27279261, -0.11074945,  0.05740409,  0.22236251],
    [-0.39525808, -0.22554661, -0.0409454,   0.14649412,  0.32397316],
    [-0.42305111, -0.24223728, -0.04287027,  0.15997045,  0.35014525],
  ],
  [
    [-0.55857474, -0.39065825, -0.19198182,  0.02378408,  0.23735671],
    [-0.27150199, -0.07088804,  0.13562939,  0.33099728,  0.50158768],
    [-0.51014825, -0.30524429, -0.06755202,  0.17806392,  0.40333043]]])
print('h greška: ', rel_error(expected_h, h))

# Vanila RNN: unazad
U fajlu `funkcije/rnn_layers.py`, implementirajte prolaz unazad za vanila RNN u funkciji `rnn_backward`. Ovo treba pokrenuti  back-propagation nad cijelom sekvencom, pozivajući `rnn_step_backward` funkciju koju ste definisali ranije. Trebali biste vidjeti greške reda `e-6` ili manje. 

In [ ]:
np.random.seed(231)

N, D, T, H = 2, 3, 10, 5

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.random.randn(H)

out, cache = rnn_forward(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, db = rnn_backward(dout, cache)

fx = lambda x: rnn_forward(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: rnn_forward(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: rnn_forward(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: rnn_forward(x, h0, Wx, Wh, b)[0]
fb = lambda b: rnn_forward(x, h0, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

print('dx greška: ', rel_error(dx_num, dx))
print('dh0 greška: ', rel_error(dh0_num, dh0))
print('dWx greška: ', rel_error(dWx_num, dWx))
print('dWh greška: ', rel_error(dWh_num, dWh))
print('db greška: ', rel_error(db_num, db))

# Word embedding: unaprijed
U sistemima dubokog učenja, često riječi predstavljamo vektorima. Svakoj riječi u vokabularu je pridružen vektor, i ovi vektori će se naučiti sa ostatkom sistema.

U fajlu `funkcije/rnn_layers.py`, implementirajte funkciju `word_embedding_forward` da konvertujete riječi (predstavljene cijelim brojevima - integers)) u vektore. Pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebalo bi da vidite grešku reda `e-8` ili manje.

In [ ]:
N, T, V, D = 2, 4, 5, 3

x = np.asarray([[0, 3, 1, 2], [2, 1, 0, 3]])
W = np.linspace(0, 1, num=V*D).reshape(V, D)

out, _ = word_embedding_forward(x, W)
expected_out = np.asarray([
 [[ 0.,          0.07142857,  0.14285714],
  [ 0.64285714,  0.71428571,  0.78571429],
  [ 0.21428571,  0.28571429,  0.35714286],
  [ 0.42857143,  0.5,         0.57142857]],
 [[ 0.42857143,  0.5,         0.57142857],
  [ 0.21428571,  0.28571429,  0.35714286],
  [ 0.,          0.07142857,  0.14285714],
  [ 0.64285714,  0.71428571,  0.78571429]]])

print('out greška: ', rel_error(expected_out, out))

# Word embedding: unazad
Implementirajte prolaz unazad za word embedding funkciju u funkciji `word_embedding_backward`. Nakon toga pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebalo biste vidjeti grešku reda `e-11` ili manje.

In [ ]:
np.random.seed(231)

N, T, V, D = 50, 3, 5, 6
x = np.random.randint(V, size=(N, T))
W = np.random.randn(V, D)

out, cache = word_embedding_forward(x, W)
dout = np.random.randn(*out.shape)
dW = word_embedding_backward(dout, cache)

f = lambda W: word_embedding_forward(x, W)[0]
dW_num = eval_numerical_gradient_array(f, W, dout)

print('dW greška: ', rel_error(dW, dW_num))

# Privremeni afajn sloj
U svakom vremenskom koraku koristimo afajn funkciju da transformišemo RNN skriveni vektor u tom vremenskom koraku u scores za svaku riječ u vokabularu. Zbog toga što je veoma slično afajn sloju koji ste implementirali, mi smo vam je obezbijedili u funkcijama `temporal_affine_forward` i `temporal_affine_backward` u fajlu `funkcije/rnn_layers.py`. Pokrenite sledeće ćelije da biste provjerili vašu implementaciju. Trebali biste vidjeti grešku reda `e-9` ili manje.

In [ ]:
np.random.seed(231)

N, T, D, M = 2, 3, 4, 5
x = np.random.randn(N, T, D)
w = np.random.randn(D, M)
b = np.random.randn(M)

out, cache = temporal_affine_forward(x, w, b)

dout = np.random.randn(*out.shape)

fx = lambda x: temporal_affine_forward(x, w, b)[0]
fw = lambda w: temporal_affine_forward(x, w, b)[0]
fb = lambda b: temporal_affine_forward(x, w, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dw_num = eval_numerical_gradient_array(fw, w, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

dx, dw, db = temporal_affine_backward(dout, cache)

print('dx greška: ', rel_error(dx_num, dx))
print('dw greška: ', rel_error(dw_num, dw))
print('db greška: ', rel_error(db_num, db))

# Privremena Softmax funkcija cilja
U RNN jezičkom modelu, u svakom vremenskom koraku dobijamo score za svaku riječ u vokabularu. Mi znamo pravu riječ u svakom trenutku, tako da koristimo softmax funkciju cilja da bismo izračunali funkciju cilja i gradijente u svakom vremenskom koraku. Sumiramo vrijednosti funkcije cilja tokom vremena i usrednjavamo je u minibatch.

Međutim postoji začkoljica: kako radimo na minibatch-evima i različiti opisi mogu imati različite dužine, dodajemo `<NULL>` tokene na kraju svakog opisa tako da svi opisi imaju istu dužinu. Ne želimo da ti `<NULL>` tokeni utiču na gradijente i funkciju cilja, pa kao dodatak score-ovima i pravim labelama naša funkcija cilja uzima i `mask` niz koji govori o tome koji elementi utiču na funkciju cilja.

Kako je ovo slično softmax funkciji cilja koju ste već implementirali, mi smo vam dali njenu implementaciju u fajlu `funkcije/rnn_layers.py` u funkciji `temporal_softmax_loss`.

Pokrenite sledeću ćeliju da biste provjerili implementaciju. Trebalo bi da vidite grešku reda `e-7` ili manje.

In [ ]:
from funkcije.rnn_layers import temporal_softmax_loss

N, T, V = 100, 1, 10

def check_loss(N, T, V, p):
    x = 0.001 * np.random.randn(N, T, V)
    y = np.random.randint(V, size=(N, T))
    mask = np.random.rand(N, T) <= p
    print(temporal_softmax_loss(x, y, mask)[0])

check_loss(100, 1, 10, 1.0)   # Treba biti oko 2.3
check_loss(100, 10, 10, 1.0)  # Treba biti oko 23
check_loss(5000, 10, 10, 0.1) # Treba biti oko 2.3

N, T, V = 7, 8, 9

x = np.random.randn(N, T, V)
y = np.random.randint(V, size=(N, T))
mask = (np.random.rand(N, T) > 0.5)

loss, dx = temporal_softmax_loss(x, y, mask, verbose=False)

dx_num = eval_numerical_gradient(lambda x: temporal_softmax_loss(x, y, mask)[0], x, verbose=False)

print('dx greška: ', rel_error(dx, dx_num))

# RNN za opisivanje slika
Sada kada ste implementirali potrebne slojeve, možete ih kombinovati da napravite model za opisivanje slika. Otvorite fajl
`funkcije/classifiers/rnn.py` i pogledajte `CaptioningRNN` klasu.

Implementirajte prolaze unaprijed i unazad modela u `loss` funkciji. Za sada je potrebno da samo implementirate u slučaju gdje je `cell_type='rnn'` za vanila RNN; kasnije ćete implementirati za LSTM. Nakon toga pokrenite sledeću ćeliju i trebalo bi da vidite grešku reda `e-10` ili manje.

In [ ]:
N, D, W, H = 10, 20, 30, 40
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 13

model = CaptioningRNN(word_to_idx,
          input_dim=D,
          wordvec_dim=W,
          hidden_dim=H,
          cell_type='rnn',
          dtype=np.float64)

for k, v in model.params.items():
    model.params[k] = np.linspace(-1.4, 1.3, num=v.size).reshape(*v.shape)

features = np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D)
captions = (np.arange(N * T) % V).reshape(N, T)

loss, grads = model.loss(features, captions)
expected_loss = 9.83235591003

print('funkcija cilja: ', loss)
print('očekivana vrijednost funkcije cilja: ', expected_loss)
print('razlika: ', abs(loss - expected_loss))

Pokrenite sledeću ćeliju da biste provjerili klasu `CaptioningRNN`; trebalo biste da vidite grešku reda `e-6` ili manje.

In [ ]:
np.random.seed(231)

batch_size = 2
timesteps = 3
input_dim = 4
wordvec_dim = 5
hidden_dim = 6
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
vocab_size = len(word_to_idx)

captions = np.random.randint(vocab_size, size=(batch_size, timesteps))
features = np.random.randn(batch_size, input_dim)

model = CaptioningRNN(word_to_idx,
          input_dim=input_dim,
          wordvec_dim=wordvec_dim,
          hidden_dim=hidden_dim,
          cell_type='rnn',
          dtype=np.float64,
        )

loss, grads = model.loss(features, captions)

for param_name in sorted(grads):
    f = lambda _: model.loss(features, captions)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[param_name], verbose=False, h=1e-6)
    e = rel_error(param_grad_num, grads[param_name])
    print('%s relativna greška: %e' % (param_name, e))

# Pretrenirajte male podatke
Slično `Solver` klasi koju smo koristili da treniramo klasifikacione modele u prethodnim nedeljama, ovdje ćemo koristiti klasu `CaptioningSolver` da treniramo modele opisivanja slika. Otvorite fajl `funkcije/captioning_solver.py` i prođite kroz `CaptioningSolver` klasu; trebalo bi da vam djeluje poznato.

Kada se upoznate sa API-jem, pokrenite sledeću ćeliju da bi vaš model overfit-ova na malom uzorku od 100 trening primjera. Na kraju biste trebali vidjeti vrijednost funkcije cilja manju od 0.1.

In [ ]:
np.random.seed(231)

small_data = load_coco_data(max_train=50)

small_rnn_model = CaptioningRNN(
          cell_type='rnn',
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          hidden_dim=512,
          wordvec_dim=256,
        )

small_rnn_solver = CaptioningSolver(small_rnn_model, small_data,
           update_rule='adam',
           num_epochs=50,
           batch_size=25,
           optim_config={
             'learning_rate': 5e-3,
           },
           lr_decay=0.95,
           verbose=True, print_every=10,
         )

small_rnn_solver.train()


plt.plot(small_rnn_solver.loss_history)
plt.xlabel('Iteracija')
plt.ylabel('Funkcija cilja')
plt.show()

# Uzorci tokom vremena testiranja
Za razliku od klasifikacionih modela, modeli opisivanja slika se ponašaju veoma različitom tokom vremena treniranja i testiranja. Tokom vremena treniranja imamo pristup pravim opisima, pa ubacamo prave riječi kao ulaze RNN u svakom vremenskom koraku. Tokom vremena testiranja, mi uzorkujemo iz distribucije nad rečnikom u svakom vremenskom koraku, i ubacujemo uzorak kao RNN u sledećem vremenskom koraku.

U fajlu `funkcije/classifiers/rnn.py`, implementirajte `sample` metod za uzorkovanje tokom vremena testiranja. Nakon toga pokrenite sledeći uzorak iz vašeg overfitted model na trening i validacionim podacima. Uzorci na trening podacima treba da budi veoma dobri; uzorci na validacionim podacima vjerovatno neće imati smisla.

In [ ]:
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(small_data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = small_rnn_model.sample(features)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        plt.imshow(image_from_url(url))
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()